In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gradio as gr

In [2]:
# --- Step 1: Text Data ---
text = """hello how are you doing today hello how are you feeling hello i am fine thank you hello how can i help you today"""

words = text.split()
vocab = sorted(set(words))
word_index = {w: i+1 for i, w in enumerate(vocab)}  # word to index
index_word = {i+1: w for i, w in enumerate(vocab)}  # index to word
vocab_size = len(vocab) + 1

# --- Step 2: Prepare sequences ---
sequences = []
for i in range(3, len(words)):
    seq = words[i-3:i+1]
    encoded = [word_index[w] for w in seq]
    sequences.append(encoded)

sequences = np.array(sequences)
x, y = sequences[:, :-1], sequences[:, -1]
x = pad_sequences(x, maxlen=3)

# --- Step 3: Build the RNN Model ---
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=10, input_length=3),
    SimpleRNN(50, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=300, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [3]:
import pickle

with open('/content/drive/MyDrive/Divya/textgen.pkl', 'wb') as file:
    pickle.dump(model,file)

In [4]:
with open('/content/drive/MyDrive/Divya/textgen.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [5]:
# --- Step 4: Text Generation Function ---
def generate_text(seed_text, next_words=5):
    for _ in range(next_words):
        token_list = [word_index.get(w, 0) for w in seed_text.split()]
        token_list = pad_sequences([token_list], maxlen=3)
        predicted = model.predict(token_list, verbose=0)
        predicted_word = index_word[np.argmax(predicted)]
        seed_text += ' ' + predicted_word
    return seed_text

# --- Step 5: Gradio Interface ---
def gradio_interface(seed_text, num_words):
    return generate_text(seed_text, next_words=num_words)

demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Enter Seed Text (e.g., 'hello how are')"),
        gr.Slider(1, 20, value=5, step=1, label="Number of Words to Generate")
    ],
    outputs="text",
    title="Text Generation using RNN",
    description="Enter initial words and generate text using a simple RNN."
)

# Launch app
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://330b26a7204e472e4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
